In [1]:
import VariationalBayes as vb
import LogisticGLMM_lib as logit_glmm
from VariationalBayes.SparseObjectives import Objective, SparseObjective

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import autograd

import copy
from scipy import optimize

import os
import json

import time

import pickle


In [2]:
# Load data saved by stan_results_to_json.R and run_stan.R in LRVBLogitGLMM.

analysis_name = 'simulated_data_small'

data_dir = os.path.join(os.environ['GIT_REPO_LOC'],
                        'LinearResponseVariationalBayes.py/Models/LogisticGLMM/data')
json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)

json_file = open(json_filename, 'r')
json_dat = json.load(json_file)
json_file.close()

stan_dat = json_dat['stan_dat']
#vp_base = json_dat['vp_base']

print(stan_dat.keys())
K = stan_dat['K'][0]
NObs = stan_dat['N'][0]
NG = stan_dat['NG'][0]
#N = NObs / NG
y_g_vec = np.array(stan_dat['y_group'])
y_vec = np.array(stan_dat['y'])
x_mat = np.array(stan_dat['x'])

glmm_par = logit_glmm.get_glmm_parameters(K=K, NG=NG)

# Define a class to contain prior parameters.
prior_par = logit_glmm.get_default_prior_params(K)
prior_par['beta_prior_mean'].set(np.array(stan_dat['beta_prior_mean']))

prior_par['beta_prior_info'].set(np.array(stan_dat['beta_prior_info']))

prior_par['mu_prior_mean'].set(stan_dat['mu_prior_mean'][0])
prior_par['mu_prior_info'].set(stan_dat['mu_prior_info'][0])

prior_par['tau_prior_alpha'].set(stan_dat['tau_prior_alpha'][0])
prior_par['tau_prior_beta'].set(stan_dat['tau_prior_beta'][0])

# An index set to make sure jacobians match the order expected by R.
prior_par_indices = copy.deepcopy(prior_par)
prior_par_indices.set_name('Prior Indices')
prior_par_indices.set_vector(np.array(range(prior_par_indices.vector_size())))

free_par_vec = glmm_par.get_free()
init_par_vec = copy.deepcopy(free_par_vec)

print(np.mean(y_vec))

dict_keys(['y', 'tau_prior_beta', 'y_group', 'K', 'mu_prior_info', 'tau_prior_alpha', 'NG', 'beta_prior_info', 'beta_prior_mean', 'N', 'x', 'mu_prior_mean'])
0.324


In [3]:
# Define moment parameters
moment_wrapper = logit_glmm.MomentWrapper(glmm_par)
get_moment_jacobian = autograd.jacobian(moment_wrapper.get_moment_vector)
moment_jac = get_moment_jacobian(init_par_vec)


model = logit_glmm.LogisticGLMM(glmm_par, prior_par, x_mat, y_vec, y_g_vec, num_gh_points=10)
model.get_e_log_prior()
model.get_log_lik()
model.get_entropy()

objective = Objective(model.glmm_par, model.get_kl)
objective.fun_free(free_par_vec)

# # PriorHess evaluates the second order derivative d2 EPrior / dpar dprior_par
def get_e_log_prior(prior_vec, free_par):
    model.glmm_par.set_free(free_par)
    model.prior_par.set_vector(prior_vec)
    return model.get_e_log_prior()

get_prior_model_grad = autograd.grad(get_e_log_prior, argnum=0)
get_prior_hess = autograd.jacobian(get_prior_model_grad, argnum=1)

import timeit

time_num = 10

num_draws = 3
model.set_gh_points(num_draws)
print('num_draws = ', num_draws)
print('\tFunction time:',
      timeit.timeit(lambda: objective.fun_free(free_par_vec), number=time_num) / time_num)

print('\tGrad time:', 
      timeit.timeit(lambda: objective.fun_free_grad(free_par_vec), number=time_num) / time_num)

print('\tHessian vector product time:',
      timeit.timeit(lambda: objective.fun_free_hvp(free_par_vec, free_par_vec + 1), number=time_num) / time_num)

prior_vec = model.prior_par.get_vector()
prior_hess_time = time.time()
get_prior_hess(prior_vec, init_par_vec)
prior_hess_time = time.time() - prior_hess_time
print('\tPrior hess time: ', prior_hess_time)


num_draws =  3
	Function time: 0.0011065320984926074
	Grad time: 0.008107000699965283
	Hessian vector product time: 0.01480242179823108
	Prior hess time:  0.06830835342407227


In [4]:
glmm_par.set_free(np.random.random(glmm_par.free_size()))
glmm_indices = copy.deepcopy(glmm_par)
glmm_indices.set_vector(np.arange(0, glmm_indices.vector_size()))

def get_local_parameters(K, beta_diag_min=0., u_info_min=0.):
    local_par = vb.ModelParamsDict('Local GLMM Parameters')
    local_par.push_param(vb.MVNParam('beta', K, min_info=beta_diag_min))
    local_par.push_param(vb.UVNParamVector('u', 1, min_info=u_info_min))
    return local_par

local_par = get_local_parameters(K)
local_indices = get_local_parameters(K)
local_indices.set_vector(np.arange(0, local_indices.vector_size()))
print(local_indices)


Local GLMM Parameters:
	beta:
beta_mean:
[0 1 2 3 4]
beta_info:
[[  5.   6.   8.  11.  15.]
 [  6.   7.   9.  12.  16.]
 [  8.   9.  10.  13.  17.]
 [ 11.  12.  13.  14.  18.]
 [ 15.  16.  17.  18.  19.]]
	u:
u_mean:
[20]
u_info:
[21]


In [49]:
import VariationalBayes.ExponentialFamilies as ef

def get_global_entropy_args(info_mu, info_beta, tau_shape, tau_rate):
    return \
        ef.univariate_normal_entropy(info_mu) + \
        ef.multivariate_normal_entropy(info_beta) + \
        ef.gamma_entropy(tau_shape, tau_rate)

def get_local_entropy_args(info_u):
    return ef.univariate_normal_entropy(info_u)

def set_local_row(glmm_par, local_par):
    local_par['beta'].set_vector(glmm_par['beta'].get_vector())

    local_par['u'].mean.set(glmm_par['u'].mean.get()[row])
    local_par['u'].info.set(glmm_par['u'].info.get()[row])


class SparseModelObjective(object):
    def __init__(self, glmm_par):
        self.glmm_par = copy.deepcopy(glmm_par)
        self.glmm_indices = copy.deepcopy(glmm_par)
        self.glmm_indices.set_vector(np.arange(0, glmm_indices.vector_size()))

        K = glmm_par['beta'].dim()
        self.local_par = get_local_parameters(K)
        self.local_indices = get_local_parameters(K)
        self.local_indices.set_vector(np.arange(0, local_indices.vector_size()))
        self.local_indices_vec = self.local_indices.get_vector()
        
    def get_global_entropy(self):
        info_mu = self.glmm_par['mu'].info.get()
        info_beta = self.glmm_par['beta'].info.get()
        tau_shape = self.glmm_par['tau'].shape.get()
        tau_rate = self.glmm_par['tau'].rate.get()
        return \
            get_global_entropy_args(info_mu, info_beta, tau_shape, tau_rate)
        
    def get_full_local_entropy(self, glmm_vec):
        self.glmm_par.set_vector(glmm_vec)
        return get_local_entropy_args(self.glmm_par['u'].info.get())

    def get_local_entropy(self, local_vec):
        self.local_par.set_vector(local_vec)
        return get_local_entropy_args(self.local_par['u'].info.get())
    
    def set_local_row(self, row):
        set_local_row(self.glmm_par, self.local_par)
        set_local_row(self.glmm_indices, self.local_indices)
        

sparse_model = SparseModelObjective(glmm_par)
NG
row = 3
get_local_entropy_hess = autograd.hessian(sparse_model.get_local_entropy)
get_full_local_entropy_hess = autograd.hessian(sparse_model.get_full_local_entropy)

sparse_model.set_local_row(row)    
#get_local_entropy_hess(sparse_model.local_par.get_vector())
full_hess = get_full_local_entropy_hess(sparse_model.glmm_par.get_vector())

# print(sparse_model.local_indices)
# print('------------')
# print(sparse_model.local_par['u'].mean.get())
#print(sparse_model.local_par)

# u_mean_index = sparse_model.local_indices['u'].mean.get()
# print(sparse_model.glmm_par.get_vector()[u_mean_index])




In [43]:
from scipy.sparse import csr_matrix

hess_vals = [] # These will be the entries of dkl / dz dweight^T
hess_rows = [] # These will be the z indices
hess_cols = [] # These will be the data indices

obs_hess_dim = sparse_model.local_par.vector_size()
full_hess_dim = sparse_model.glmm_par.vector_size()

for row in range(NG):
    sparse_model.set_local_row(row)    
    full_indices = sparse_model.local_indices.get_vector()
    row_hess_val = get_local_entropy_hess(sparse_model.local_par.get_vector())

    for row in range(obs_hess_dim):
        for col in range(obs_hess_dim):
            if row_hess_val[row, col] != 0:
                hess_vals.append(row_hess_val[row, col])
                hess_rows.append(int(full_indices[row]))
                hess_cols.append(int(full_indices[col]))


sparse_hess = csr_matrix((hess_vals, (hess_rows, hess_cols)),
                         (full_hess_dim, full_hess_dim))


  (124, 124)	0.0709064592585
  (125, 125)	0.30843128106
  (126, 126)	0.170395171435
  (127, 127)	0.0762675194972
  (128, 128)	0.0793623607453
  (129, 129)	0.233580961547
  (130, 130)	0.109760102242
  (131, 131)	0.238864040393
  (132, 132)	0.263165679478
  (133, 133)	0.128640724095
  (134, 134)	0.0826709790661
  (135, 135)	0.341343549446
  (136, 136)	0.457760551703
  (137, 137)	0.19545263501
  (138, 138)	0.167708023358
  (139, 139)	0.188791973066
  (140, 140)	0.155220492036
  (141, 141)	0.125624892418
  (142, 142)	0.189375479719
  (143, 143)	0.168896862176
  (144, 144)	0.0725012452164
  (145, 145)	0.393225646206
  (146, 146)	0.123275392336
  (147, 147)	0.472770313778
  (148, 148)	0.212410482812
  :	:
  (199, 199)	0.384092350634
  (200, 200)	0.110579407801
  (201, 201)	0.248077506781
  (202, 202)	0.0926683535621
  (203, 203)	0.324188337609
  (204, 204)	0.233475968546
  (205, 205)	0.285096667609
  (206, 206)	0.480392770738
  (207, 207)	0.187559260645
  (208, 208)	0.114189723575
  (209, 20

In [57]:
np.max(np.abs(full_hess - sparse_hess))
#plt.plot(full_hess[:], sparse_hess.todense()[:], 'k.')

0.0